## Import needed libraries

In [8]:
#import json
import os
import sys
#from getpass import getpass


import boto3
#import cassio

#from langchain.embeddings import BedrockEmbeddings
#from langchain.llms import Bedrock
#from langchain.vectorstores import Cassandra
#from langchain.schema import Document
#from langchain.prompts import PromptTemplate
#from langchain.document_loaders import TextLoader

from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

## Astra DB Setup

In [4]:
from dotenv import load_dotenv

load_dotenv()
ASTRA_DB_ID = os.environ["ASTRA_DB_ID"]
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]
ASTRA_VECTOR_ENDPOINT = os.environ["ASTRA_VECTOR_ENDPOINT"]
ASTRA_DB_KEYSPACE = "blueillusion"
ASTRA_DB_COLLECTION = "catalogue"
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_PROJECT"] = "blueillusion"
os.environ["LANGCHAIN_TRACING_V2"] = "false"

## AWS Credentials Setup

These are set as environment variables for usage by the subsequent `boto3` calls. Please refer to [boto3's documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html) on the possible ways to supply your credentials in a more production-like environment.

In [5]:
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]

## Set up AWS Bedrock objects

#### Chat

In [9]:
chat = BedrockChat(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={"temperature": 0.1},
)

In [10]:
messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
chat(messages)

Voici la traduction en français : J'adore programmer.

AIMessage(content="Voici la traduction en français : J'adore programmer.")

#### Embedding

In [4]:
bedrock_runtime = boto3.client("bedrock-runtime", "us-west-2")
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=bedrock_runtime
    )

## Set up the Vector Store

This command will create a suitable table in your database if it does not exist yet:

In [ ]:
vector_store = Cassandra(
    embedding=bedrock_embeddings,
    table_name="shakespeare_act5",
    session=None,  # <-- meaning: use the global defaults from cassio.init()
    keyspace=None,  # <-- meaning: use the global defaults from cassio.init()
)